In [2]:
import os
import pandas as pd

def exportar_medication_request_para_modeled(diretorio_curated, diretorio_modeled, arquivo_saida, chunksize):
    arquivos_parquet = [f for f in os.listdir(diretorio_curated) if f.startswith('lote_') and f.endswith('.parquet')]
    
    if not arquivos_parquet:
        print("🚨 Nenhum arquivo Parquet encontrado no diretório.")
        return
    
    os.makedirs(diretorio_modeled, exist_ok=True)
    
    lista_medications = []

    for arquivo in arquivos_parquet:
        caminho_arquivo = os.path.join(diretorio_curated, arquivo)
        print(f"🔄 Lendo arquivo: {caminho_arquivo}")

        # ✅ Lê o arquivo inteiro (sem chunksize)
        df = pd.read_parquet(caminho_arquivo, engine='pyarrow')

        # ✅ Divide em chunks manualmente
        total_registros = len(df)
        for i in range(0, total_registros, chunksize):
            chunk = df.iloc[i:i + chunksize].reset_index(drop=True)

            # ✅ Filtrar as entradas onde o resourceType é 'MedicationRequest'
            df_medication_request = chunk[chunk['resource_resourceType'] == 'MedicationRequest']
            
            # ✅ Agrupar por 'resource_medicationCodeableConcept_text' e contar as ocorrências
            medication_counts = (df_medication_request
                                 .groupby('resource_medicationCodeableConcept_text')
                                 .size()
                                 .reset_index(name='count'))
            
            lista_medications.append(medication_counts)

    # ✅ Concatena todos os lotes
    if lista_medications:
        df_final = pd.concat(lista_medications, ignore_index=True)
        
        # ✅ Agrupar novamente para consolidar os resultados finais
        df_final = (df_final
                    .groupby('resource_medicationCodeableConcept_text', as_index=False)
                    .sum()
                    .sort_values(by='count', ascending=False)
                    .head(10))
        
        # ✅ Resetar índice
        df_final = df_final.reset_index(drop=True)
        
        # ✅ Salva como Excel
        caminho_saida = os.path.join(diretorio_modeled, arquivo_saida)
        df_final.to_excel(caminho_saida, index=False)
        print(f"✅ Arquivo de saída salvo em: {caminho_saida}")
    else:
        print("🚨 Nenhum dado de 'MedicationRequest' encontrado para exportar.")

# 🔥 Exemplo de uso:
diretorio_curated = r'C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\medicalrequest'
diretorio_modeled = r'C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\modeled'
arquivo_saida = 'top_medications.xlsx'
chunksize = 1000

# ✅ Executa o processo de exportação
exportar_medication_request_para_modeled(diretorio_curated, diretorio_modeled, arquivo_saida, chunksize)

🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\medicalrequest\lote_001.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\medicalrequest\lote_002.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\medicalrequest\lote_003.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\medicalrequest\lote_004.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\medicalrequest\lote_005.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\medicalrequest\lote_006.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\medicalrequest\lote_007.parquet
🔄 Lendo arquivo: C:\Users\Leonardo\OneDrive\Documentos\Git\teste_engenheiro\eng_dados\curated\medicalrequest\lote_008.